In [ ]:
import os
import sys
sys.path.append(os.path.abspath("..")) # set working dir

from web3 import Web3
from src.config import ALCHEMY_API_KEY

ETHEREUM_RPC_URL = f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(ETHEREUM_RPC_URL))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block) # latest block

# Setup

In [ ]:
from src.utils.retrieveAbi import save_abi_to_file
from src.config import MAINNET_UNISWAP_V2_ROUTER_02, MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET

# Load ABIs
save_abi_to_file(MAINNET_UNISWAP_V2_ROUTER_02) # uniswap v2 router abi
save_abi_to_file(MAINNET_SUSHISWAP_ROUTER_ADDRESS) # sushiswap router abi

# Contract Interactions

In [ ]:
import time
from src.utils.retrieveAbi import load_abi

def get_amounts_out(token1_address, token2_address, amount_in, router_contract):
    """Uses w3 getAmountsOut function to return amounts out, given two tokens, router contract and amount in"""
    token1_address = Web3.to_checksum_address(token1_address)
    token2_address = Web3.to_checksum_address(token2_address)
    path = [token1_address, token2_address]
    amounts_out = router_contract.functions.getAmountsOut(amount_in, path).call()
    return w3.from_wei(amounts_out[-1], 'ether')

In [ ]:
# uniswap
uniswap_router_abi = load_abi(MAINNET_UNISWAP_V2_ROUTER_02, CHAINID_MAINNET)
uniswap_router_address = Web3.to_checksum_address(MAINNET_UNISWAP_V2_ROUTER_02)
uniswap_router_contract = w3.eth.contract(address=uniswap_router_address, abi=uniswap_router_abi)
uniswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
uniswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
uniswap_amount_in = w3.to_wei(1, "ether")

# sushiswap
sushiswap_router_abi = load_abi(MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET)
sushiswap_router_address = Web3.to_checksum_address(MAINNET_SUSHISWAP_ROUTER_ADDRESS)
sushiswap_router_contract = w3.eth.contract(address=sushiswap_router_address, abi=sushiswap_router_abi)
sushiswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
sushiswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
sushiswap_amount_in = w3.to_wei(1, "ether")

try:
    prev_block=0
    while True:
        block = w3.eth.block_number
        uniswap_amount_out = get_amounts_out(uniswap_weth_address, uniswap_dai_address, uniswap_amount_in, uniswap_router_contract)
        sushiswap_amount_out = get_amounts_out(sushiswap_weth_address, sushiswap_dai_address, sushiswap_amount_in, sushiswap_router_contract)
        if block > prev_block:
            print(f"Block {block} | Uniswap: 1 ETH ≈ {uniswap_amount_out} DAI | Sushiswap: 1 ETH ≈ {sushiswap_amount_out} | Delta = {uniswap_amount_out-sushiswap_amount_out}")
            prev_block = block
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped by user.")

# Binance API

In [1]:
import pandas as pd

# Load the CSV
df = pd.read_csv("/home/tobias/personal-dex-trading/out/data/order_book.csv", index_col="time")
df

,bids,asks
time,,
10963965124,"[['3809.87000000', '15.51040000'], ['3809.8100...","[['3809.88000000', '8.71200000'], ['3810.07000..."
10963965361,"[['3810.06000000', '20.62370000'], ['3810.0000...","[['3810.07000000', '12.90080000'], ['3810.1300..."
10963965448,"[['3810.06000000', '11.33270000'], ['3810.0000...","[['3810.07000000', '14.71520000'], ['3810.1300..."
10963965471,"[['3810.06000000', '11.63080000'], ['3810.0000...","[['3810.07000000', '14.71520000'], ['3810.1300..."
10963965566,"[['3810.06000000', '20.92580000'], ['3810.0100...","[['3810.07000000', '4.26200000'], ['3810.13000..."
...,...,...
10964008850,"[['3809.97000000', '10.90080000'], ['3809.9000...","[['3809.98000000', '11.65910000'], ['3809.9900..."
10964009296,"[['3809.98000000', '13.89120000'], ['3809.9000...","[['3809.99000000', '15.28460000'], ['3810.0000..."
10964009462,"[['3809.98000000', '18.36810000'], ['3809.9600...","[['3809.99000000', '13.69720000'], ['3810.0000..."


Next, use SBE Market data for lower latency: https://developers.binance.com/docs/binance-spot-api-docs/sbe-market-data-streams

# Unichain

In [ ]:
from src.uniswap.v2_client import UniswapV2Client
from src.config import UNICHAIN_WETH, UNICHAIN_USDC

uniswap_client = UniswapV2Client()

uniswap_client.get_amounts_out(UNICHAIN_WETH, UNICHAIN_USDC)

In [ ]:
import os
import sys
sys.path.append(os.path.abspath("..")) # set working dir

from src.utils.retrieveAbi import save_abi_to_file, load_abi
from src.config import (
    UNICHAIN_UNIVERSAL_ROUTER_V2_ADDRESS, CHAINID_UNICHAIN,
    UNICHAIN_UNISWAP_V2_ROUTER_02, UNICHAIN_WETH, UNICHAIN_USDC,
    ALCHEMY_UNICHAIN_BASE_RPC_URL)

save_abi_to_file(UNICHAIN_UNIVERSAL_ROUTER_V2_ADDRESS, chain_id=CHAINID_UNICHAIN)
save_abi_to_file(UNICHAIN_UNISWAP_V2_ROUTER_02, chain_id=CHAINID_UNICHAIN)

In [ ]:
WETH_AMOUNT_IN = 0.01
USDC_AMOUNT_IN = 38
weth_wei_amount_in = w3.to_wei(WETH_AMOUNT_IN, "ether")
usdc_amounts_in = USDC_AMOUNT_IN * 10 ** 6 # USDC has 6 decimals

router_address = Web3.to_checksum_address(UNICHAIN_UNISWAP_V2_ROUTER_02)
router_abi = load_abi(router_address, CHAINID_UNICHAIN)
router_contract = w3.eth.contract(address=router_address, abi=router_abi)

alchemy_rpc_url = f"{ALCHEMY_UNICHAIN_BASE_RPC_URL}{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(alchemy_rpc_url))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block.number) # latest block number

In [ ]:
amounts_out = router_contract.functions.getAmountsOut(weth_wei_amount_in, [UNICHAIN_WETH, UNICHAIN_USDC]).call()
print(f"Input Amount (WETH): {WETH_AMOUNT_IN} WETH")
print(f"Output Amount (USDC): {amounts_out[-1] / (10 ** 6)} USDC")  # USDC has 6 decimals
print(f"----")
amounts_out_reverse = router_contract.functions.getAmountsOut(usdc_amounts_in, [UNICHAIN_USDC, UNICHAIN_WETH]).call()
print(f"Input Amount (USDC): {USDC_AMOUNT_IN} USDC")
print(f"Output Amount (WETH): {w3.from_wei(amounts_out_reverse[-1], "ether")} WETH ")  # USDC has 6 decimals

Next, we can either do:
- arbitrage between Uniswap v2 and Binance 


- arbitrage between Uniswap v2 and v4 pools